In [1]:
import numpy as np
import tensorflow as tf
import datetime
import data_helpers
import train_cnn
from sent_cnn import SentCNN
import config
%load_ext autoreload
%autoreload 2

In [2]:
x_u_i, x_r_i, y, max_len, U = train_cnn.load_data(config.config)

In [ ]:
train_cnn.train_cnn(x_u_i, x_r_i, y, max_len, U, config.config,debug=False)

DEBUG: embedded_u -> Tensor("embedding/embedding_lookup:0", shape=(?, 22, 100), dtype=float32)
DEBUG: embedded_r -> Tensor("embedding/embedding_lookup_1:0", shape=(?, 6, 22, 100), dtype=float32)
DEBUG: embedded_u_expanded -> Tensor("embedding/ExpandDims:0", shape=(?, 22, 100, 1), dtype=float32)
DEBUG: embedded_r_expanded -> Tensor("embedding/ExpandDims_1:0", shape=(?, 6, 22, 100, 1), dtype=float32)
DEBUG: pooled_outputs_u -> [<tf.Tensor 'conv-maxpool-3-u/pool-u:0' shape=(?, 1, 1, 4) dtype=float32>, <tf.Tensor 'conv-maxpool-4-u/pool-u:0' shape=(?, 1, 1, 4) dtype=float32>]
DEBUG: h_pool_u -> Tensor("concat:0", shape=(?, 1, 1, 8), dtype=float32)
DEBUG: h_pool_flat_u -> Tensor("Reshape:0", shape=(?, 1, 8), dtype=float32)
DEBUG: pooled_outputs_r -> [<tf.Tensor 'conv-maxpool-3-u/concat:0' shape=(?, 1, 6, 4) dtype=float32>, <tf.Tensor 'conv-maxpool-4-u/concat:0' shape=(?, 1, 6, 4) dtype=float32>]
DEBUG: h_pool_r -> Tensor("concat_1:0", shape=(?, 1, 6, 8), dtype=float32)
DEBUG: h_pool_flat_r -